# Курсовая. Бот

## <a id='Задание'>Задание</a>

- Продумать тему курсового проекта.
- Подготовить данные для обучения.
- Продумать интенты на которые вы будете обучаться.

**Для сдачи задания необходимо написать**

- на какую тему бот будет общаться (на каких датасетах планируете обучать модели),
- на какие интенты вы будете обучать бота.

**Из обязательных (всего обязательно 3-ри интента):**

- ваш бот дожен работать в режиме болталка
- на ваш выбор

## <a id='Интенты'>Интенты</a>
1. Болталка: чат-бот ведет диалог в стиле Игры престолов.
2. Игра "Угадай персонажа"
3. Генератор тем докладов.
4. Прогноз погоды.
5. Анализ отзыва о работе бота.

In [2]:
! pip install pymorphy2
! pip install annoy
! pip install stop_words
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 98.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=6407b5566eb74fb33aa261d1733aa808f1fef8a794adbd4e9b6cb362a1baf7bb
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=583977 sha256=375bb0af0e556a01f1c3907877289020f304ab1151a728ec07070b11d20888ed
  Stored in directory: /root/.c

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import string
import io
import chardet
import os
import codecs

# import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
import annoy
# import gensim.models

from stop_words import get_stop_words
import nltk
import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

import time
import wikipedia
wikipedia.set_lang("ru")

from sklearn.model_selection import train_test_split
from string import punctuation
from sklearn.preprocessing import LabelEncoder

import re
import annoy
import pickle

import tqdm
from tqdm import tqdm_notebook

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer

In [5]:
# Преобразование данных

morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
exclude = set(string.punctuation)
noise = set(sw + punkt)

## <a id='Болталка'>Болталка в стиле Игры престолов</a>

In [4]:
# Файл Игра престолов
file = '1_igra-prestolov_utf.txt'

In [10]:
# Загружаем данные

with open(Path(DATA_PATH).joinpath(file), encoding='utf-8') as f:
    data = f.read()

In [11]:
# Загружаем данные и сразу преобразуем их в токены
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

# Предложения вопросы
tokens_q = tokenizer.tokenize(data)
print(len(tokens_q)), print(tokens_q[:3])

6021
['Джордж Мартин\nИгра престолов\nПосвящается Мелинде\nПролог\n–\xa0Надо бы поворачивать,\xa0– встревожился Гаред, как только лес вокруг них начал темнеть.', '– Одичалые мертвы.', '–\xa0Неужели ты боишься покойников?']


(None, None)

In [12]:
# Предложения ответы: из получившегося списка вопросов формируем ответы -
# каждое последующее предложение является ответом для предыдущего

tokens_a = tokens_q[1:]
tokens_a.append(tokens_q[:1])
print(len(tokens_a)), print(tokens_a[:3])

6021
['– Одичалые мертвы.', '–\xa0Неужели ты боишься покойников?', '– вопросил сир Уэймар Ройс с легким намеком на улыбку.']


(None, None)

In [13]:
df = pd.DataFrame({'question': tokens_q, 'answer': tokens_a})
df

,question,answer
0,Джордж Мартин\nИгра престолов\nПосвящается Мел...,– Одичалые мертвы.
1,– Одичалые мертвы.,– Неужели ты боишься покойников?
2,– Неужели ты боишься покойников?,– вопросил сир Уэймар Ройс с легким намеком на...
3,– вопросил сир Уэймар Ройс с легким намеком на...,Гаред не попался на крючок.
4,Гаред не попался на крючок.,"За свои пятьдесят лет он успел навидаться, как..."
...,...,...
6016,– Здесь и далее примеч. пер.\n\n\n\n\n2\n\n\nА...,3\n\n\nСвободное владение.
6017,3\n\n\nСвободное владение.,"4\n\n\n1 стоун = 14 фунтов, около 50 кг.\n\n\n..."
6018,"4\n\n\n1 стоун = 14 фунтов, около 50 кг.\n\n\n...","6\n\n\nстаринная английская мера длины, около ..."
6019,"6\n\n\nстаринная английская мера длины, около ...",7\n\n\nоколо 200 м.


In [14]:
# Функция преобразования текста

def preprocess_txt(text):
    return [t for t in [morpher.parse(token)[0].normal_form
                                for token in word_tokenize(text)] if t not in noise]


def start_prep(data):
    result = []

    for text in tqdm(data):
      prep = preprocess_txt(text)
      result.append(prep)

    return result

In [15]:
# Процедура поиска ближайших вопросов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for question in df.question:
        n_model = 0
        index_map[counter] = question
        que_prep = preprocess_txt(question)

        vector_model = np.zeros(300) #вектор твита

        for word in que_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1

        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор=

        model_index.add_item(counter, vector_model) #добавляем элемент для индексации

        counter += 1

        if counter > 100000:
            break

    model_index.build(100) #из индексов создаем кластеры
    return model_index, index_map

In [16]:
# Функция поиска ближайших соседей
def get_response(que, index, model, index_map, df):
    token = preprocess_txt(que)
    vector = np.zeros(300)
    norm = 0

    for word in token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_que = index.get_nns_by_vector(vector, 15, include_distances=True)

    result = pd.DataFrame({'question': [index_map[i]  for i in similar_que[0]],
                           'distance' : similar_que[1], 'answer': [df[df.index == i].answer.values[0] for i in similar_que[0]]})


    return result

In [17]:
tokens = start_prep(df.question)
modelFT = FastText(sentences=tokens, vector_size=300, min_count=1, window=5, workers=8)
ft_index, index_map = get_model_index(modelFT.wv)

100%|██████████| 6021/6021 [00:16<00:00, 367.14it/s]


Пример работы модели

In [18]:
smp = 'Тирион?'

print(smp)
# get_response(smp, ft_index, modelFT.wv, index_map, df)

# single answer
rand = random.randint(0,14)
resp = get_response(smp, ft_index, modelFT.wv, index_map, df)
print(resp.answer.values[rand])

Тирион?
Лорд-командующий выделил ему комнаты в Королевской башне, носившей это имя, несмотря на то что ни один король не посещал ее целый век.


## <a id='Угадай_персонажа'>Игра "Угадай персонажа"</a>

Пользователь вводит описание персонажа, модель делает предсказание, к кому из персонажей это описание может относиться.

In [19]:
# В качестве данных возьмем wiki описание персонажей Властелина колец
heroes = ['Тирион', 'Бран', 'Дейенерис', 'Эддард (Нед) Старк', 'Джейме', 'Серсея', 'Кейтилин',
          'Джон Сноу', 'Арья', 'Санса', 'Петир Бейлиш', 'Варис']

heroes_data = {'description': [],
               'character': []}

for hero in tqdm(heroes):
  wiki = wikipedia.search(hero)
  ans = wikipedia.page(wiki[0]).content
  heroes_data['description'].append(ans)
  heroes_data['character'].append(hero)

100%|██████████| 12/12 [00:13<00:00,  1.12s/it]


In [20]:
df_heroes = pd.DataFrame.from_dict(heroes_data)
df_heroes.head(2)

,description,character
0,Ти́рион Ла́ннистер (англ. Tyrion Lannister) — ...,Тирион
1,"Замок Бран (рум. Castelul Bran, нем. Törzburg,...",Бран


In [21]:
# Преобразование данных
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
cats = set(df_heroes.character.str.lower().to_list()) # исключаем из обучающих текстов имена самих персонажей
morpher = MorphAnalyzer()

def preprocess_text(txt):
    #print('raw: ', txt)
    txt = str(txt)
    #print('to str: ', txt)
    txt = txt.lower()
    #print('lower: ', txt)
    txt = ''.join([i for i in txt if not i.isdigit()])
    #print('no digits: ', txt)
    #txt = ''.join([i for i in txt if i.isalpha()])
    txt = re.sub('[()!@#$"«»/]', '', txt)
    #print('no sym: ', txt)
    txt = "".join(c for c in txt if c not in exclude)
    #print('exclude: ', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split(' ') if word not in sw]
    #print('morpher: ', txt)
    txt = " ".join(c for c in txt if c not in cats)
    #print('wo cats: ', txt)
    return "".join(txt)

In [22]:
tqdm.pandas()
df_heroes['description'] = df_heroes['description'].progress_apply(preprocess_text)

100%|██████████| 12/12 [00:03<00:00,  3.97it/s]


In [23]:
df_heroes['description'].sample(2)

3    э́ддард нед старк  — персонаж роман американск...
1    замок браный рум castelul bran törzburg венг t...
Name: description, dtype: object

In [24]:
# Проведем токенизацию по 3-ем словам
upd_df2 = {'description': [],
           'character': []}

for char in df_heroes.character.unique():
  data = df_heroes[df_heroes.character == char].description.to_list()
  for d in data:
    desc = tokenizer.tokenize(d)
    for el in desc:
      #print(el)
      el = el.replace('—', ' ')
      el = re.sub('[a-zA-Z]', '', el)
      words = el.split(' ')
      words = [word for word in words if word != '']
      words = [word for word in words if len(word) >= 3]
      #print(words)
      n_min = 0
      n_max = len(words)

      while n_min != n_max-2:
        tok = [''.join(word) for word in words[n_min:n_min+3]]
        n_min += 1
        #print(tok)

        upd_df2['description'].append(preprocess_text(tok))
        upd_df2['character'].append(char)

In [25]:
df2 = pd.DataFrame.from_dict(upd_df2)
df2.sample(2)

,description,character
3218,открытый чёрный рыба,Джейме
4897,королевский гавань наёмник,Серсея


In [26]:
df2.shape

(12720, 2)

In [27]:
df2 = df2.drop_duplicates(subset=['description'], keep='first')
df2.shape

(11964, 2)

In [28]:
# Разбиваем на трейн и тест
df_train, df_test = train_test_split(df2, test_size=0.10, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((10767, 2), (1197, 2))

In [29]:
df_train['description'].sample(5)

9239                  лианный старк мать
2408           прозвище мизинец являться
5390    поединок потерпеть предсказуемый
179                   тирания брат стать
2163             выпить получать задание
Name: description, dtype: object

In [30]:
text_corpus_train = df_train['description'].values
text_corpus_test = df_test['description'].values

In [31]:
# Определяем токенайзер
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\]^_`{|}~\t\n',
                     lower = False, split = ' ')

# Обучаем токенайзер на тренировочном наборе данных - > получаем готовый словарь
tokenizer.fit_on_texts(text_corpus_train)

In [32]:
# На основе полученного словаря токены переводятся в их id и сформировываем последовательности sequences

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [33]:
# Всего слов в словаре
word_count = len(tokenizer.index_word) + 1

# Находим максимальную длину будущей последовательности в тренировочном наборе
training_length = max([len(i.split()) for i in text_corpus_train])

word_count, training_length

(3729, 3)

In [34]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

X_train[5]

array([681, 402, 132], dtype=int32)

In [35]:
# Преобразование значений y
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['character'])
test_enc_labels = le.transform(df_test['character'])
le.classes_

array(['Арья', 'Бран', 'Варис', 'Дейенерис', 'Джейме', 'Джон Сноу',
       'Кейтилин', 'Петир Бейлиш', 'Санса', 'Серсея', 'Тирион',
       'Эддард (Нед) Старк'], dtype=object)

In [36]:
num_classes = df_train['character'].nunique()
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

In [37]:
y_train.shape, y_train[3]

((10767, 12),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [38]:
%%time
early_stopping=EarlyStopping(monitor='val_loss', patience = 1, mode = 'min')

# RNN - > CNN
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=num_classes,
              trainable=True,
              mask_zero=True))

model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3, padding = 'same', activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=25,
                    epochs=40,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/40
431/431 [==============================] - 10s 10ms/step - loss: 0.3438 - accuracy: 0.1298 - val_loss: 0.2727 - val_accuracy: 0.1763
Epoch 2/40
431/431 [==============================] - 4s 9ms/step - loss: 0.2842 - accuracy: 0.1768 - val_loss: 0.2500 - val_accuracy: 0.2222
Epoch 3/40
431/431 [==============================] - 6s 13ms/step - loss: 0.2537 - accuracy: 0.2552 - val_loss: 0.2285 - val_accuracy: 0.3183
Epoch 4/40
431/431 [==============================] - 4s 9ms/step - loss: 0.2242 - accuracy: 0.3661 - val_loss: 0.2034 - val_accuracy: 0.4177
Epoch 5/40
431/431 [==============================] - 4s 9ms/step - loss: 0.1965 - accuracy: 0.4753 - val_loss: 0.1798 - val_accuracy: 0.5280
Epoch 6/40
431/431 [==============================] - 5s 13ms/step - loss: 0.1708 - accuracy: 0.5739 - val_loss: 0.1627 - val_accuracy: 0.6007
Epoch 7/40
431/431 [==============================] - 4s 10ms/step - loss: 0.1522 - accuracy: 0.6333 - val_loss: 0.1489 - val_accuracy: 0.6349
E

In [39]:
predictions = model.predict(X_test)

38/38 [==============================] - 1s 4ms/step


In [40]:
preds = []
for pred in predictions:
    preds.append(pred.argmax())

len(preds), preds[:3]

(1197, [8, 8, 11])

In [41]:
dec_preds = le.inverse_transform(preds)
dec_preds[:3]

array(['Санса', 'Санса', 'Эддард (Нед) Старк'], dtype=object)

In [42]:
cols = ['description', 'character']
vals = df_test[cols]
vals['preds'] = dec_preds
vals.sample(5)

,description,character,preds
447,встречаться кротовый,Санса,Санса
601,помочь попросить старое,Джон Сноу,Джон Сноу
1149,вместе тело мирцелла,Серсея,Джейме
523,выбрать покои хостер,Кейтилин,Кейтилин
913,указ класть,Серсея,Арья


Пример работы модели

In [43]:
def prep_input_text(text):
  text = preprocess_text(text)
  text_list = []
  text_list.append(text)
  sequences = tokenizer.texts_to_sequences(text_list)
  pad = pad_sequences(sequences, maxlen=training_length)
  return pad

def make_pred(pad):
  predictions_single = model.predict(pad)
  preds = [predictions_single.argmax()]
  return le.inverse_transform(preds)

In [44]:
inp = 'интриган'
print('Загадайте героя: ', inp)

pred = make_pred(prep_input_text(inp))
print('Ответ: ', pred)

Загадайте героя:  интриган
1/1 [==============================] - 0s 40ms/step
Ответ:  ['Эддард (Нед) Старк']


## <a id='Генератор_имен'>Генератор имен Песни Льда и Пламени</a>

Пользователю предлагается задать начальную букву имени, бот в ответ выдаст сгенерированное имя в стиле Песни льда и пламени. Для этой задачи напишем простой алгоритм, который будет основываться на списке имен из Игры престолов и генерировать случайное.

In [45]:
names = set(['Эддард', 'Кейтилин', 'Робб', 'Санса', 'Арья', 'Бран',
             'Рикон', 'Бенджен', 'Лианна', 'Теон', 'Русе', 'Рамси',
             'Ходор', 'Оша', 'Жойен', 'Мира', 'Жиенна', 'Дейенерис',
             'Визерис', 'Рейегар', 'Эйегон', 'Эйерис', 'Джорах',
             'Бринден', 'Миссандея', 'Даарио', 'Тайвин', 'Серсея',
             'Джейме', 'Джоффри', 'Мирцелла', 'Тирион', 'Томмен',
             'Киван', 'Лансель', 'Бронн', 'Григор', 'Сандор', 'Подрик',
             'Станнис', 'Селиса', 'Ширен', 'Мелисандра', 'Давос', 'Ренли',
             'Бриенна', 'Берик', 'Джендри', 'Бейлон', 'Аша', 'Эурон',
             'Петир', 'Варис', 'Виктарион', 'Эйерон', 'Доран', 'Арианна',
             'Элия', 'Оберин', 'Эллария', 'Арео', 'Хостер', 'Эдмур',
             'Бринден', 'Уолдер', 'Мейс', 'Лорас', 'Маргери', 'Оленна',
             'Рендилл', 'Джиор', 'Эйемон', 'Йорен', 'Сэмвелл', 'Янос',
             'Аллисер', 'Манс', 'Игритт', 'Боуэн', 'Эддисон', 'Тормунд',
             'Пицель', 'Барристан', 'Арис', 'Илин', 'Квиберн', 'Дрого',
             'Сирио', 'Якен', 'Иллирио', 'Торос', 'Гиздар', 'Йеццан',
             'Тихо', 'Мерибальд', 'Юнелла'
             ])

In [46]:
df_names = pd.DataFrame(list(names), columns=['name'])

In [47]:
# Разобъем имена на фонемы и создадим словарь по первым буквам

fons = {'а': [], 'б': [], 'в': [], 'г': [], 'д': [], 'е': [], 'ё': [], 'ж': [], 'з': [], 'и': [], 'й': [], 'к': [], 'л': [], 'м': [], 'н': [],
        'о': [], 'п': [], 'р': [], 'с': [], 'т': [], 'у': [], 'ф': [], 'х': [], 'ц': [], 'ч': [], 'ш': [], 'щ': [], 'ы':[], 'ъ':[], 'ь': [], 'э': [], 'ю': [], 'я': [],
        '-': []}

for name in df_names.name.str.lower().to_list():
  #print(name, len(name))
  n_min = 0
  n_max = len(name)
  while n_min != n_max:
    fon = name[n_min:n_min+3]
    fl = fon[:1]
    fons[fl].append(fon)
    n_min +=1

In [48]:
def gen_name(letter):
  letter = letter.lower()
  vars = fons[letter]
  r = random.randint(0, len(vars))
  name = vars[r]
  name_length = random.randint(1, 3)

  while name_length:

    try:
      nl = random.choice(list(fons.keys()))[0]
      var_nl = fons[nl]
      r_nl = random.randint(0, len(var_nl))
      name += var_nl[r_nl]
    except IndexError:
      pass

    name_length -= 1

  return name.title()

In [49]:
let = 'о'
print('Ваше имя: ', gen_name(let))

Ваше имя:  Ормл
